In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
import re
from datetime import datetime
import time

In [2]:
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
         'accept-encoding': 'gzip, deflate, br',
         'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
         'accept-language': 'en-US,en;q=0.9',
         'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}

### Latitude and Longitudes of known places
Purva Skywood : 12.8933808, 77.6609237

In [8]:
##getting the list of the restaurants
start = time.time()
arr1=[[]]
lat = 12.8933808
long = 77.6609237
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
r1_js = r1.json()
tot_rest_cnt = r1_js['data']['totalSize']

for off in range(0, tot_rest_cnt, 14):
    api_url = 'https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING'
    r1=requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
    r1_js = r1.json()    
    try:
        rn = len(r1_js['data']['cards'])
    except:
        break
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']/100
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']/100
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']
            #making an array of the above
            arr = np.array([api_url, name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug])
            try:
                arr1 = np.append(arr1, [arr], axis=0)
            except:
                arr1 = [arr]
columns=['api_url', 'name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
         'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
df_rest_list = pd.DataFrame(arr1, columns=columns)
df_rest_list = df_rest_list.drop_duplicates(['name', 'area'], ignore_index=True)
now=datetime.now()
fname = now.strftime('%Y%m%d%H%M') + '_purva.csv'
os.chdir('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\Outputs')
df_rest_list.to_csv(fname, index= False)

end =time.time()
print('Time taken:', end-start)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Time taken: 57.498411655426025


In [9]:
tot_rest_cnt

823

## Restaurant Menu

In [ ]:
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=12.8933808&lng=77.6609237&collection=46147&offset=800&pageType=COLLECTION&type=rcv2&page_type=DESKTOP_COLLECTION_LISTING', headers= headers)

In [ ]:
r1.json()

In [ ]:
df_rest_list

In [ ]:
rest_slug = df_rest_list['slug'][1] + '-' + df_rest_list['rest_id'][1]
rest_url = 'https://www.swiggy.com/restaurants/' + rest_slug
mr = requests.get(rest_url, headers = headers)

In [ ]:
mr_text = re.split('window.webpackManifest', re.split('INITIAL_STATE__ = ', mr.text)[1])[0].strip().replace(';', '')
mr_json = json.loads(mr_text)

In [ ]:
for item in list(dict.keys(mr_json['menu']['items'])):
    item_id = mr_json['menu']['items'][item]['id']
    name = mr_json['menu']['items'][item]['name']
    category = mr_json['menu']['items'][item]['category']
    stock = mr_json['menu']['items'][item]['inStock']
    price = mr_json['menu']['items'][item]['price']/100
    vegClassifier = mr_json['menu']['items'][item]['attributes']['vegClassifier']
    menu_temp = np.array([category, name, stock, price, vegClassifier, item_id])
    try:
        menu = np.append(menu, [menu_temp], axis=0)
    except:
        menu = [menu_temp]

In [ ]:
df_rest_list

In [ ]:
#putting together the fragments of code for the full menu
start = time.time()
for i in range(len(df_rest_list)):
    rest_slug = df_rest_list['slug'][i] + '-' + df_rest_list['rest_id'][i]
    rest_url = 'https://www.swiggy.com/restaurants/' + rest_slug
    rest_name = df_rest_list['name'][i]
    
    mr = requests.get(rest_url, headers = headers)
    mr_text = re.split('window.webpackManifest', re.split('INITIAL_STATE__ = ', mr.text)[1])[0].strip().replace(';', '')
    mr_json = json.loads(mr_text)

    for item in list(dict.keys(mr_json['menu']['items'])):
        item_id = mr_json['menu']['items'][item]['id']
        name = mr_json['menu']['items'][item]['name']
        category = mr_json['menu']['items'][item]['category']
        stock = mr_json['menu']['items'][item]['inStock']
        price = mr_json['menu']['items'][item]['price']/100
        try:
            vegClassifier = mr_json['menu']['items'][item]['attributes']['vegClassifier']
        except:
            vegClassifier =''
        menu_temp = np.array([rest_name, category, name, stock, price, vegClassifier, item_id])
        try:
            menu = np.append(menu, [menu_temp], axis=0)
        except:
            menu = [menu_temp]
end=time.time()
print('Time taken:', end-start)

In [ ]:
menu

In [ ]:
mr_json['menu']['items'][item]

# This is for finding the place suggestions 

In [ ]:
place_api = 'https://www.swiggy.com/dapi/misc/places-autocomplete?input=P&types='

In [ ]:
requests.get(place_api).json()

In [ ]:
lat_long_api = 'https://www.swiggy.com/dapi/misc/reverse-geocode?place_id=ChIJMVbVAVsTrjsRAOFkz2Up-gE'


In [ ]:
js=requests.get(lat_long_api).json()

In [ ]:
js['data']

In [ ]:
#after getting the lat long from above
some_api = 'https://www.swiggy.com/dapi/pop/check?lat=26.25636&lng=82.526328'

In [ ]:
requests.get(some_api).text